[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/QuantLet/EMQA/blob/main/EMQA_energy_overview/EMQA_energy_overview.ipynb)

# EMQA_energy_overview

Overview of major energy commodity prices (2020-2025).

**Output:** `energy_markets_overview.pdf`

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

plt.rcParams.update({
    'figure.facecolor': 'none',
    'axes.facecolor': 'none',
    'savefig.facecolor': 'none',
    'savefig.transparent': True,
    'axes.grid': False,
    'axes.spines.top': False,
    'axes.spines.right': False,
    'font.size': 11,
    'figure.figsize': (12, 6),
})

COLORS = {
    'blue': '#1A3A6E', 'red': '#CD0000', 'green': '#2E7D32',
    'orange': '#E67E22', 'purple': '#8E44AD', 'gray': '#808080',
    'cyan': '#00BCD4', 'amber': '#B5853F'
}

def save_fig(fig, name):
    fig.savefig(name, bbox_inches='tight', transparent=True, dpi=300)
    print(f"Saved: {name}")


In [ ]:
import yfinance as yf

def fetch(ticker, start='2020-01-01', end='2025-12-31'):
    d = yf.download(ticker, start=start, end=end, progress=False)
    if isinstance(d.columns, pd.MultiIndex):
        return d['Close'].squeeze().dropna()
    return d['Close'].dropna()


In [ ]:
tickers = {'Brent Oil': 'BZ=F', 'WTI Oil': 'CL=F', 'Natural Gas': 'NG=F', 'Heating Oil': 'HO=F'}
data = {k: fetch(v) for k, v in tickers.items()}
df = pd.DataFrame(data).dropna()

fig, axes = plt.subplots(2, 2, figsize=(14, 9))
colors = [COLORS['blue'], COLORS['red'], COLORS['green'], COLORS['orange']]

for ax, (name, col), c in zip(axes.flat, df.items(), colors):
    ax.plot(df.index, col, color=c, linewidth=0.8)
    ax.axhline(col.mean(), color='gray', ls='--', alpha=0.5, lw=0.8)
    ax.set_title(name, fontweight='bold')
    ax.set_ylabel('USD')

handles = [plt.Line2D([0],[0], color=c, lw=2) for c in colors]
fig.legend(handles, list(df.columns), loc='upper center', bbox_to_anchor=(0.5, -0.01), ncol=4, frameon=False)
fig.suptitle('Energy Commodity Prices (2020-2025)', fontsize=14, fontweight='bold')
fig.tight_layout(rect=[0, 0.03, 1, 0.97])
save_fig(fig, 'energy_markets_overview.pdf')
plt.show()
